# Computing volume fractions by intersecting unstructured meshes

## Research data

### Source code 

[git repostitory](link)

[git tag](link) 

[source code archive](link)

### Article / report 

[DOI / arXivID](link)

### Data 

[secondary data](link)

[primary data](link)


In [ ]:
# Pick up .csv files 
import os
import pandas as pd
from math import pi
from plot_study import *

In [ ]:
import warnings 
# Set the GEOM_VOF_INIT environmental variable to the folder where you want the data to be saved. 
data_dir = os.curdir
try:
    data_dir = os.environ["GEOM_VOF_INIT"] 
except:
    warnings.warn("No GEOM_VOF_INIT environmental variable set, storing data in the current working directory.") 

## Exact volumes for sphere and ellipsoid
**Warning**: The values in the cell below need to match those used in the parameter study.

In [ ]:
# Exact volume of the sphere.  
sphere_radius = 0.15 #  WARNING: Adapt this if another radius is chosen in templateCase/sphere.geo.template 
sphere_volume = sphere_radius**3 * 4 * pi / 3.

# Exact volume of the ellipsoid.  
a = 0.4 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 
b = 0.3 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 
c = 0.2 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 
ellipsoid_volume = 4 * a * b * c * pi / 3. 

## SMCI tests

In [ ]:
for study_pattern in ["sphere", "spherePerturbed"]:
    data = plot_study(study_pattern, alg_name="SMCI", exact_volume=sphere_volume, data_write_dir=data_dir)
    
for study_pattern in ["ellipsoid", "ellipsoidPerturbed"]:
    data = plot_study(study_pattern, alg_name="SMCI", exact_volume=ellipsoid_volume, data_write_dir=data_dir)

## SMCA tests

In [ ]:
for study_pattern in ["sphereCartesian", "spherePerturbed"]:
    data = plot_study(study_pattern, alg_name="SMCA", exact_volume=sphere_volume, data_write_dir=data_dir, csv_file="smcaVofInit.csv")
    
for study_pattern in ["ellipsoidCartesian", "ellipsoidPerturbed"]: 
    data = plot_study(study_pattern, alg_name="SMCA", exact_volume=ellipsoid_volume, data_write_dir=data_dir, csv_file="smcaVofInit.csv")

plot_smca_refinement_convergence("smcaRefinementConvergence", exact_volume=sphere_volume, data_write_dir=data_dir)